In [14]:
import pdfplumber
import pandas as pd
import datetime as dt
import sys
from itables import show
import re


In [49]:
pdf = pdfplumber.open('pdf/kemen atr bpn.pdf')

In [50]:
df_tabel_rinci_fix = pd.DataFrame(columns = ['No Peserta', 'Nama', 'Tanggal Lahir', 'Kode Pendidikan', 'IPK', 
                            'TWK', 'TIU', 'TKP', 'Total SKD', 'Skor SKD', 'Total SKB', 
                            'Skor SKB', 'Nilai Akhir', 'Keterangan', 'Pendidikan', 'Instansi', 
                            'Jabatan', 'Lokasi Formasi', 'Jumlah Formasi', 'Jenis Formasi']
                   )
df_tabel_rinci_fix

,No Peserta,Nama,Tanggal Lahir,Kode Pendidikan,IPK,TWK,TIU,TKP,Total SKD,Skor SKD,Total SKB,Skor SKB,Nilai Akhir,Keterangan,Pendidikan,Instansi,Jabatan,Lokasi Formasi,Jumlah Formasi,Jenis Formasi


In [51]:
print("Mulai")
for i in range(3, 10): #2204
    start_time = dt.datetime.now()
    tabel = pdf.pages[i].extract_tables()
    df_tabel_rinci = tabel[int(len(tabel)) - 1]  # Indeks 2 untuk tabel ketiga
    # Ubah tabel menjadi DataFrame
    df_tabel_rinci = pd.DataFrame(df_tabel_rinci)
    #cek apakah tabel per orangan atau bukan
    if df_tabel_rinci.shape[1] != 15:
        #print('Skip ' + str(i))
        continue
    df_tabel_rinci = df_tabel_rinci.iloc[4:, 1:]
    df_tabel_rinci.columns = ['No Peserta', 'Nama', 'Tanggal Lahir', 'Kode Pendidikan', 'IPK', 
                            'TWK', 'TIU', 'TKP', 'Total SKD', 'Skor SKD', 'Total SKB', 
                            'Skor SKB', 'Nilai Akhir', 'Keterangan']
    
    teks = pdf.pages[i].extract_text_simple()
    # Regex untuk mengekstrak informasi program studi
    pattern = re.compile(r'\((\d{7})\) ([A-Z-1]+(?: [A-Z-1]+)*[\w\s]+?)(?=\s*/|\s*$)', re.MULTILINE)
    matches = pattern.findall(teks)

    # Menyusun hasil ekstraksi
    program_studies = [f"({match[0]}) {match[1].strip()}" for match in matches]

    # Menghapus entri yang tidak diinginkan, seperti "Pendidikan 1"
    program_studies = [study for study in program_studies if 'Pendidikan 1' not in study]

    program_studies_cleaned = [study.replace('\n', ' ') for study in program_studies]

    # Menggabungkan nilai dalam program_studies_cleaned menjadi teks tunggal dengan spasi sebagai pemisah
    program_studies_text = ' / '.join(program_studies_cleaned)

    program_studies_text = re.sub(r'\d+$', '', program_studies_text).strip()
    program_studies_text = re.sub(r' Halaman \d+ dari \d+ halaman', '', program_studies_text)
    
    if len(program_studies_text) > 0:
        match_instansi = re.search(r'Instansi\s*:\s*([^\n]*)', teks)
        if match_instansi:
            instansi = match_instansi.group(1).strip()
        else:
            instansi = "Tidak ada hasil yang cocok"
            
        parts_instansi = instansi.split(' - ')
        instansi = parts_instansi[1].rsplit(' ', 1)
        instansi = instansi[0]

        match_jab = re.search(r'Jabatan\s*:\s*([^\n]*)', teks)
        if match_jab:
            jabatan = match_jab.group(1).strip()
        else:
            jabatan = "Tidak ada hasil yang cocok"

        parts_jabatan = jabatan.split(' - ')
        jabatan = parts_jabatan[1].rsplit(' ', 1)
        jabatan = jabatan[0]
        
        match_lokasi_formasi = re.search(r'Lokasi Formasi\s*:\s*([^\n]*)', teks)
        if match_lokasi_formasi:
            lokasi_formasi_cek = match_lokasi_formasi.group(1).strip()
        else:
            lokasi_formasi_cek = "Tidak ada hasil yang cocok"

        def cek_lokasi_formasi(teks):
            # Regex untuk menangkap semua teks setelah tanda '-'
            match = re.search(r' - (.+)', teks)
            if match:
                return match.group(1)
            return None
        
        parts = lokasi_formasi_cek.split(' - ')
        kode = parts[0]

        #formasi_parts = parts[1].rsplit(' ', 1)
        lokasi_formasi = cek_lokasi_formasi(lokasi_formasi_cek)

        def cek_jumlah_formasi(teks):
            match = re.search(r' (\d+)$', teks)
            if match:
                return match.group(1)
            return None
        jumlah_formasi = cek_jumlah_formasi(lokasi_formasi_cek)


        def menentukan_jenis_formasi(teks):
            match_jenis_formasi = re.search(r'Jenis Formasi : \S+ - (\w+)', teks)
            if match_jenis_formasi:
                return match_jenis_formasi.group(1)
            return "Tidak ada hasil yang cocok"

        jenis_formasi = menentukan_jenis_formasi(teks)
            
    else:
        instansi = instansi
        jabatan = jabatan
        lokasi_formasi = lokasi_formasi
        jumlah_formasi = jumlah_formasi
        jenis_formasi = jenis_formasi
    
    df_tabel_rinci['Instansi'] = instansi
    df_tabel_rinci['Jabatan'] = jabatan
    df_tabel_rinci['Lokasi Formasi'] = lokasi_formasi
    df_tabel_rinci['Jumlah Formasi'] = jumlah_formasi
    df_tabel_rinci['Pendidikan'] = program_studies_text
    df_tabel_rinci['Jenis Formasi'] = jenis_formasi
    
    
    df_tabel_rinci_fix = pd.concat([df_tabel_rinci_fix, df_tabel_rinci], ignore_index=True)
    
    #print('Berhasil Halaman ' + str(i))
    if i%100 == 99:
        curr_time = dt.datetime.now()
        print('Selesai hingga halaman ' + str(i) + '-' + str(curr_time - start_time))
        start_time = dt.datetime.now()
print("Selesai")
    

Mulai
Selesai


In [53]:
#df_tabel_rinci_fix = df_tabel_rinci_fix.drop(columns=['Nama'])
df_tabel_rinci_fix

,No Peserta,Nama,Tanggal Lahir,Kode Pendidikan,IPK,TWK,TIU,TKP,Total SKD,Skor SKD,Total SKB,Skor SKB,Nilai Akhir,Keterangan,Pendidikan,Instansi,Jabatan,Lokasi Formasi,Jumlah Formasi,Jenis Formasi
0,2140242110012626,DIO BIAN PRIATAMA,30-08-1994,4417093,3.430,110,140,193,443,32.218,56.000,33.600,65.818,P/L,(4417093) D-III TEKNIK INFORMATIKA / (4450070)...,Kementerian Agraria dan Tata Ruang/Badan Perta...,VERIFIKATOR BERKAS PERMOHONAN HAK,KEMENTERIAN ATR/BPN - SUMATERA UTARA 6,6,UMUM
1,2140242120014868,SRI MEIYANTI SITORUS,02-05-2000,4417093,3.490,95,105,195,395,28.727,50.000,30.000,58.727,P/L,(4417093) D-III TEKNIK INFORMATIKA / (4450070)...,Kementerian Agraria dan Tata Ruang/Badan Perta...,VERIFIKATOR BERKAS PERMOHONAN HAK,KEMENTERIAN ATR/BPN - SUMATERA UTARA 6,6,UMUM
2,2140242120017847,DHESY KHALLA LESTARI,12-12-1993,4417093,3.390,90,110,179,379,27.564,49.000,29.400,56.964,P/L,(4417093) D-III TEKNIK INFORMATIKA / (4450070)...,Kementerian Agraria dan Tata Ruang/Badan Perta...,VERIFIKATOR BERKAS PERMOHONAN HAK,KEMENTERIAN ATR/BPN - SUMATERA UTARA 6,6,UMUM
3,2140242120017863,RIZKA NOVITAWANTY\nNASUTION,22-11-1995,4417093,3.490,95,115,170,380,27.636,45.000,27.000,54.636,P/L,(4417093) D-III TEKNIK INFORMATIKA / (4450070)...,Kementerian Agraria dan Tata Ruang/Badan Perta...,VERIFIKATOR BERKAS PERMOHONAN HAK,KEMENTERIAN ATR/BPN - SUMATERA UTARA 6,6,UMUM
4,2140242120017851,MELDA SITINJAK,19-05-1992,4417093,3.320,70,120,186,376,27.346,43.000,25.800,53.146,P/L,(4417093) D-III TEKNIK INFORMATIKA / (4450070)...,Kementerian Agraria dan Tata Ruang/Badan Perta...,VERIFIKATOR BERKAS PERMOHONAN HAK,KEMENTERIAN ATR/BPN - SUMATERA UTARA 6,6,UMUM
5,2140242110001437,ARIEF MUHARMAN,29-04-1998,4417093,3.340,75,120,188,383,27.854,42.000,25.200,53.054,P/L,(4417093) D-III TEKNIK INFORMATIKA / (4450070)...,Kementerian Agraria dan Tata Ruang/Badan Perta...,VERIFIKATOR BERKAS PERMOHONAN HAK,KEMENTERIAN ATR/BPN - SUMATERA UTARA 6,6,UMUM
6,2140242110016031,SURYA DHARMA,06-11-1994,4417093,3.430,70,80,190,340,24.727,45.000,27.000,51.727,P/TL,(4417093) D-III TEKNIK INFORMATIKA / (4450070)...,Kementerian Agraria dan Tata Ruang/Badan Perta...,VERIFIKATOR BERKAS PERMOHONAN HAK,KEMENTERIAN ATR/BPN - SUMATERA UTARA 6,6,UMUM
7,2140242120017111,INTAN SHAVIRA,22-02-1995,4417093,3.390,80,105,190,375,27.273,35.000,21.000,48.273,P/TL,(4417093) D-III TEKNIK INFORMATIKA / (4450070)...,Kementerian Agraria dan Tata Ruang/Badan Perta...,VERIFIKATOR BERKAS PERMOHONAN HAK,KEMENTERIAN ATR/BPN - SUMATERA UTARA 6,6,UMUM
8,2140242110001502,OCTA BERRY SILABAN,12-10-1991,4417093,3.130,70,95,180,345,25.091,38.000,22.800,47.891,P/TL,,Kementerian Agraria dan Tata Ruang/Badan Perta...,VERIFIKATOR BERKAS PERMOHONAN HAK,KEMENTERIAN ATR/BPN - SUMATERA UTARA 6,6,UMUM
9,2140242120001293,KRISTINA BR GINTING,20-05-1992,4417093,3.440,80,135,170,385,28.000,32.000,19.200,47.200,P/TL,,Kementerian Agraria dan Tata Ruang/Badan Perta...,VERIFIKATOR BERKAS PERMOHONAN HAK,KEMENTERIAN ATR/BPN - SUMATERA UTARA 6,6,UMUM


In [6]:
df_tabel_rinci_fix.to_excel("hasil/atr-bpn-1200.xlsx")

# batas bawah

In [16]:
#ekstrak tabel
tabel = pdf.pages[37].extract_tables()
print(len(tabel))
df_tabel_rinci = tabel[int(len(tabel)) - 1]  # Indeks 2 untuk tabel ketiga
# Ubah tabel menjadi DataFrame
df_tabel_rinci = pd.DataFrame(df_tabel_rinci)
df_tabel_rinci = df_tabel_rinci.iloc[4:, 1:]
df_tabel_rinci.columns = ['No Peserta', 'Nama', 'Tanggal Lahir', 'Pendidikan', 'IPK', 
                          'TWK', 'TIU', 'TKP', 'Total SKD', 'Skor SKD', 'Total SKB', 
                          'Skor SKB', 'Nilai Akhir', 'Keterangan']
df_tabel_rinci

3


,No Peserta,Nama,Tanggal Lahir,Pendidikan,IPK,TWK,TIU,TKP,Total SKD,Skor SKD,Total SKB,Skor SKB,Nilai Akhir,Keterangan
4,2140202120001157,TANTRI DEA ARTIKA,01-07-1997,5100170,3.490,95,150,187,432,31.418,84.000,50.400,81.818,P/L
5,2140202120001990,ARIFAH AZIMATUN NIQMAH\nARFAT HIDAYATULLOH,07-05-1996,5100170,3.590,115,145,189,449,32.654,54.000,32.400,65.054,P/TL
6,2140202110000504,MUHAMMAD RAMADHANI,09-01-1997,5100917,3.670,100,100,195,395,28.727,55.000,33.000,61.727,P/TL


In [16]:
import re

teks = pdf.pages[38].extract_text_simple()
# Regex untuk mengekstrak informasi program studi
pattern = re.compile(r'\((\d{7})\) ([A-Z-1]+(?: [A-Z-1]+)*[\w\s]+?)(?=\s*/|\s*$)', re.MULTILINE)
matches = pattern.findall(teks)

# Menyusun hasil ekstraksi
program_studies = [f"({match[0]}) {match[1].strip()}" for match in matches]

# Menghapus entri yang tidak diinginkan, seperti "Pendidikan 1"
program_studies = [study for study in program_studies if 'Pendidikan 1' not in study]

program_studies_cleaned = [study.replace('\n', ' ') for study in program_studies]

# Menggabungkan nilai dalam program_studies_cleaned menjadi teks tunggal dengan spasi sebagai pemisah
program_studies_text = ' '.join(program_studies_cleaned)

program_studies_text = re.sub(r'\d+$', '', program_studies_text).strip()

# Menampilkan hasil
program_studies_text

'(5101272) S-1 KOMUNIKASI (5101328) S-1 MANAJEMEN KOMUNIKASI (5153000) S-1 ILMU KOMUNIKASI (5192799) S-1 HUBUNGAN MASYARAKAT SKB'

In [22]:
match_jenis_formasi = re.search(r'Jenis Formasi\s*:\s*([^\n]*)', teks)
if match_jenis_formasi:
    jenis_formasi = match_jenis_formasi.group(1).strip()
else:
    jenis_formasi = "Tidak ada hasil yang cocok"
    
parts_jenis_formasi = jenis_formasi.split(' - ')
jenis_formasi = parts_instansi[1].rsplit(' ', 1)
jenis_formasi = instansi[0]
match_jenis_formasi

<re.Match object; span=(275, 301), match='Jenis Formasi : U - UMUM 1'>

In [24]:
# Fungsi untuk mengambil nilai dari teks
def ambil_nilai(teks):
    match_jenis_formasi = re.search(r'Jenis Formasi : \S+ - (\w+)', teks)
    if match_jenis_formasi:
        return match_jenis_formasi.group(1)
    return None

# Ambil nilai dari teks
nilai1 = ambil_nilai(teks)
nilai1

'UMUM'

In [24]:
match_instansi = re.search(r'Instansi\s*:\s*([^\n]*)', teks)
if match_instansi:
    instansi = match_instansi.group(1).strip()
else:
    instansi = "Tidak ada hasil yang cocok"
    
parts_instansi = instansi.split(' - ')
instansi = parts_instansi[1].rsplit(' ', 1)
instansi = instansi[0]

match_jab = re.search(r'Jabatan\s*:\s*([^\n]*)', teks)
if match_jab:
    jabatan = match_jab.group(1).strip()
else:
    jabatan = "Tidak ada hasil yang cocok"

parts_jabatan = jabatan.split(' - ')
jabatan = parts_jabatan[1].rsplit(' ', 1)
jabatan = jabatan[0]

match_lokasi_formasi = re.search(r'Lokasi Formasi\s*:\s*([^\n]*)', teks)
if match_lokasi_formasi:
    lokasi_formasi = match_lokasi_formasi.group(1).strip()
else:
    lokasi_formasi = "Tidak ada hasil yang cocok"

parts_lokasi_formasi = lokasi_formasi.split(' - ')
lokasi_formasi_rsplit = parts_lokasi_formasi[1].rsplit(' ', 1)
lokasi_formasi = lokasi_formasi_rsplit[0]
lokasi_formasi = re.sub(r'^\d+\.\s*', "", lokasi_formasi)
jumlah_formasi = lokasi_formasi_rsplit[1]
    
print(jumlah_formasi)

1


In [24]:
df_tabel_rinci['Instansi'] = instansi
df_tabel_rinci['Jabatan'] = jabatan
df_tabel_rinci['Lokasi Formasi'] = lokasi_formasi
df_tabel_rinci['Jumlah Formasi'] = jumlah_formasi

In [10]:
match = re.search(r'Instansi\s*:\s*([^\n]*)', teks)
if match:
    result = match.group(1).strip()
    print(result)
else:
    print("Tidak ada hasil yang cocok")

3010 - Kementerian Pendidikan dan Kebudayaan 10447


In [8]:
match = re.search(r'Jabatan\s*:\s*([^\n]*)', teks)
if match:
    result = match.group(1).strip()
    print(result)
else:
    print("Tidak ada hasil yang cocok")

10101003 - ANALIS LAPORAN AKUNTABILITAS KINERJA 1


In [18]:
teks

'PANITIA SELEKSI NASIONAL PENGADAAN CPNS 2021\nREKAP HASIL INTEGRASI SKD DAN SKB\nPENGADAAN CPNS 2021\nKode Jumlah\nInstansi : 4020 - Badan Kependudukan dan Keluarga Berencana Nasional 315\nJabatan : 10106012 - ANALIS KEUANGAN 8\nLokasi Formasi : 40200018 - Maluku 1\nJenis Formasi : U - UMUM 1\nPendidikan (5100170) S-1 AKUNTANSI / (5100208) S-1 MANAJEMEN / (5100917) S-1 EKONOMI 1\nSKB\nJumlah Jumlah\nJumlah Peserta Hasil Lulus\nPeserta Metode\nFormasi Akhir\nSKB SKB\nTidak Tidak\nTotal Hadir TMS TMS-1 APS\nHadir Lulus\n1 2 3 4 5 6 7 8 9 10 11\n3 1 0 3 3 0 2 0 0 0 1\nLaporan digenerate secara otomatis melalui Aplikasi Pengolahan Nilai SSCASN Badan Kepegawaian Negara, Panitia Seleksi Nasional © 2021 39 / 503'

In [25]:
match_lokasi_formasi = re.search(r'Lokasi Formasi\s*:\s*([^\n]*)', teks)
if match_lokasi_formasi:
    lokasi_formasi_cek = match_lokasi_formasi.group(1).strip()
else:
    lokasi_formasi_cek = "Tidak ada hasil yang cocok"

# parts_lokasi_formasi = lokasi_formasi_cek.split(' - ')
# lokasi_formasi_rsplit = parts_lokasi_formasi[1].rsplit(' ', 1)
# lokasi_formasi = lokasi_formasi_rsplit[0]
# lokasi_formasi = re.sub(r'^\d+\.\s*', "", lokasi_formasi)
# jumlah_formasi = lokasi_formasi_rsplit[1]

parts = lokasi_formasi_cek.split(' - ')
kode = parts[0]
formasi_parts = parts[1].rsplit(' ', 1)
formasi = formasi_parts[0]
jumlah_formasi = formasi_parts[1]

print(kode)
print(formasi)
print(jumlah_formasi)
df = pd.DataFrame({
    'KODE': [kode],
    'FORMASI': [formasi],
    'JUMLAH FORMASI': [jumlah_formasi]
})
df

40200018
Maluku
1


,KODE,FORMASI,JUMLAH FORMASI
0,40200018,Maluku,1


In [59]:
match = re.search(r'Lokasi Formasi\s*:\s*([^\n]*)', teks)
if match:
    result = match.group(1).strip()
    print(result)
else:
    print("Tidak ada hasil yang cocok")

30101893 - 1


In [14]:
'''
Ini adalah script untuk mengubah pdf lampiran hasil CPNS jadi csv
Caveat: kalo langsung prosess 20rb halaman, agak keselek di tengah, jadi mending di-iterate beberapa halaman
Saya yakin ada cara yang lebih elegan. Feel free to give me any suggestions

Cara run 
python export_data_to_csv.py index_halaman_start index_halaman_end

Misal
python export_data_to_csv.py 0 100
'''

import pdfplumber
import pandas as pd
import datetime as dt
import sys

def check_for_detail_tables(page):
    '''
    Fungsi untuk mengecek keberadaan tabel perorangan
    Input: page (ex: pdf.pages[0])
    Output: 
        - found (binary, iya tidaknya sebuah halaman punya tabel perorangan)
        - df_returned (tabel perorangan jika ada)
    '''
    found = False
    df_returned = pd.DataFrame()
    for table in page.extract_tables():
        df = pd.DataFrame(table)
        if (df.shape[1] == 11) & (df.shape[0]>16):
            found = True
            df_returned = df
    return found, df_returned

def check_for_jabatan(page):
    '''
    Fungsi untuk mengecek keberadaan informasi lowongan
    Input: page (ex: pdf.pages[0])
    Output: 
        - dicitionary berisi informasi lowongan
    '''
    if "Lokasi Formasi :" in page.extract_text():
        text = page.extract_text()
        pendidikan = page.extract_tables()[1][0][1]
        jabatan_strings = text.split("Jabatan : ")[1].split("Lokasi")[0]
        lokasi_front = ""
        jabatan = jabatan_strings.split("\n")[0]

        if len(jabatan_strings.split("\n")) > 1:
            lokasi_front = " ".join(jabatan_strings.split("\n")[1:])
        lokasi_string = text.split("Lokasi Formasi : ")[1].split("Jenis")[0]
        if len(lokasi_string.split("\n")) > 2:
            lokasi_back = " ".join(lokasi_string.split("\n")[1:])
        else:
            lokasi_back = lokasi_string.split(" - ")[1]

        return {
            "kode_jabatan": jabatan.split(" - ")[0],
            "jabatan": jabatan.split(" - ")[1],
            "kode_lokasi": lokasi_string.split(" - ")[0],
            "lokasi_formasi": lokasi_front+lokasi_back,
            "jenis_formasi": text.split("Jenis Formasi : ")[1].split("\n")[0],
            "pendidikan": pendidikan
        }
    else:
        return {}

def get_info_from_table(df_):
    '''
    Fungsi untuk mengekstrak informasi dari tabel perorangan 
    Input: df_ (dataframe tabel perorangan)
    Output: 
        - dicitionary berisi informasi perorangan yang telah diekstrak
    '''
    skd = df_.iloc[7:10,[1,3]]
    skb = df_.iloc[13:,[1,3,4,5,6]]

    skd_dict = skd.set_index(1)[3].to_dict()
    skb_dict = skb.set_index(1)[3].to_dict()

    bobot_skb = skb.set_index(1)[5]
    bobot_skb.index = ["bobot_"+x for x in bobot_skb.index]
    bobot_skb = bobot_skb.to_dict()

    final_skb = skb.set_index(1)[6]
    final_skb.index = ["final_"+x for x in final_skb.index]
    final_skb = final_skb.to_dict()

    base_data =  {
        "no_peserta": df_.iloc[1,1],
        "kode_pendidikan": df_.iloc[1,2],
        "tanggal_lahir": df_.iloc[1,8],
        "ipk": df_.iloc[1,10],
        "keterangan": df_.iloc[7,10],
        "total_skd": df_.iloc[7,5],
        "total_skd_skala_100": df_.iloc[7,7],
        "total_skd_dengan_bobot": df_.iloc[7,8],
        "total_skd": df_.iloc[13,7],
        "total_skb_dengan_bobot": df_.iloc[13,8],
        "total_nilai_akhir": df_.iloc[7,9]
    }

    return {**base_data, **skd_dict, **skb_dict, **bobot_skb, **final_skb}

def split_df(df_):
    '''
    Fungsi untuk split tabel perorangan. Kadang ada satu halaman dengan lebih dari satu tabel.
    Input: df_ (dataframe tabel perorangan)
    Output: 
        - list berisi beberapa dataframe untuk tiap individu
    '''
    dfs = []
    header_indexes = list(df_[df_[1]=="No Peserta"].index)
    header_indexes.append(len(df_))
    for i in range(len(header_indexes)-1):
        splitted_df = df_.iloc[header_indexes[i]: header_indexes[i+1], :]
        splitted_df.index = range(len(splitted_df))
        dfs.append(splitted_df)
    return dfs


In [16]:

start_index = 1
end_index = 1000
export_filename = "cpns_"+str(start_index)+"_"+str(end_index)+".csv"
pdf = pdfplumber.open('LAMPIRAN 2.pdf')

# start iterating
result = []
last_jabatan = {}

start_time = dt.datetime.now()
for i in range(start_index, end_index):
    pg = pdf.pages[i]

    # jika halaman tsb ada info tentang lowongan, simpan
    current_jabatan = check_for_jabatan(pg)
    if current_jabatan != {}:
        last_jabatan = current_jabatan
    
    is_detail_found, detail_df = check_for_detail_tables(pg)
    # jika ketemu ada tabel perorangan
    if is_detail_found:
        splitted_df = split_df(detail_df)
        for df_ in splitted_df:
            details = get_info_from_table(df_)
            if current_jabatan == {}:
                # kalo ada info lowongan di halaman yang sama, pakai info lowongan tsb
                details.update(last_jabatan)
            else:
                # kalo ga, pake info lowongan terakhir
                details.update(current_jabatan)
                last_jabatan = current_jabatan
            result.append(details)

    # untuk logging
    if i%100 == 99:
        curr_time = dt.datetime.now()
        print("Done for "+str(i), curr_time-start_time)
        start_time = dt.datetime.now()

res = pd.DataFrame(result)
res.to_csv(export_filename)

Done for 99 0:00:17.951152
Done for 199 0:00:15.112134
Done for 299 0:00:16.784073
Done for 399 0:00:14.669551
Done for 499 0:00:16.100282
Done for 599 0:00:18.558845
Done for 699 0:00:10.093506
Done for 799 0:00:19.641069
Done for 899 0:00:11.123924
Done for 999 0:00:22.704919
